# Лабораторная работа №1: Introduction to Apache Spark.<br>Выполнил студент группы 6133-010402D Мелешенко Иван.

# Задание на лабораторноую работу


##[Текст задания на лабораторную работу](https://gitlab.com/ssau.tk.courses/big_data/-/blob/master/L1%20-%20Introduction%20to%20Apache%20Spark/L1_Apache_Spark_Tasks.md)

Для данных велопарковок Сан-Франциско (trips.csv, stations.csv):
1. Найти велосипед с максимальным временем пробега.
2. Найти наибольшее геодезическое расстояние между станциями.
3. Найти путь велосипеда с максимальным временем пробега через станции.
4. Найти количество велосипедов в системе.
5. Найти пользователей потративших на поездки более 3 часов.

# Этап 1. Подготовка средств и данных.

## Шаг 1. Установка PySpark

PySpark — это API Apache Spark, применяемый для распределенной обработки больших данных.

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 59.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=5111f1989369ea5ddfb5034d279609048ac35ad497cfddd5e46eb31da5b22480
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


## Шаг 2. Импорт необходимых библоитек.

### ***SparkContext***

SparkContext — это точка входа в любую функциональность Spark. Когда запускаем любое приложение Spark, запускается программа драйвера, которая выполняет основную функцию, и здесь запускается SparkContext. Затем программа драйвера запускает операции внутри исполнителей на рабочих узлах.

### ***SparkConf***

Конфигурация приложения Spark. Используется для установки различных параметров Spark в виде пар ключ-значение. В большинстве случаев объект SparkConf создается с помощью new SparkConf(), который будет загружать значения из любого файла spark.* Системные свойства Java также устанавливаются в приложении. В этом случае параметры, которые установленны непосредственно на объекте SparkConf, имеют приоритет над системными свойствами.

### Импорт библиотек

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from typing import NamedTuple
from datetime import datetime
from functools import reduce

## Шаг 3. Создание проекта.

### Создание точки конфигурации, установка имени проекта, и выбора подключения к кластеру


In [3]:
sc = SparkContext(conf=SparkConf().setAppName('L1ItroductionToApacheSpark').setMaster("local[*]"))

### Проверка установки

In [4]:
sc

<SparkContext master=local[*] appName=L1ItroductionToApacheSpark>

## Шаг 4. Загрузка данных из файлов.

Для ускорения загрузки файлов на гугл коллаб воспользуемся гугл диском.

In [5]:
from google.colab import drive
drive.mount('/content/drive')
data_trip = sc.textFile("/content/drive/MyDrive/trips.csv")
data_station = sc.textFile("/content/drive/MyDrive/stations.csv")

Mounted at /content/drive


# Этап 2. Предобработка данных.

## Шаг 1. Определение заголовка данных

Определение заголовка необходимо провести с целью его дальнейшего отделения от данных.

In [6]:
headerOfTrips = data_trip.first()
headerOfStations = data_station.first()

## Шаг 2. Фильтрация данных.

### 2.1. Произведем отделение заголовка, определенного на предыдущем шаге.

In [7]:
tripsWithOutHeader = data_trip.filter(lambda row: row != headerOfTrips)
stationsWithOutHeader = data_station.filter(lambda row: row != headerOfStations)

### 2.2. Просмотр полученного результата

In [8]:
tripsWithOutHeader.take(10)

['4576,63,,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127',
 '4607,,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138',
 '4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214',
 '4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060',
 '4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103',
 '4927,103,8/29/2013 18:54,Golden Gate at Polk,59,8/29/2013 18:56,Golden Gate at Polk,59,527,Subscriber,94109',
 '4500,109,8/29/2013 13:25,Santa Clara at Almaden,4,8/29/2013 13:27,Adobe on Almaden,5,679,Subscriber,95112',
 '4563,111,8/29/2013 14:02,San Salvador at 1st,8,8/29/2013 14:04,San Salvador at 1st,8,687,Subscriber,95112',
 '4760,113,8/29/2013 17:01,South Van Ness at Market,66,8/29/2013 17:03,South Van Ness at Market,66,553,Subscriber,9410

In [9]:
stationsWithOutHeader.take(10)

['2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013',
 '3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013',
 '4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013',
 '5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013',
 '6,San Pedro Square,37.336721000000004,-121.894074,15,San Jose,8/7/2013',
 '7,Paseo de San Antonio,37.333798,-121.88694299999999,15,San Jose,8/7/2013',
 '8,San Salvador at 1st,37.330165,-121.88583100000001,15,San Jose,8/5/2013',
 '9,Japantown,37.348742,-121.89471499999999,15,San Jose,8/5/2013',
 '10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013',
 '11,MLK Library,37.335885,-121.88566000000002,19,San Jose,8/6/2013']

### 2.3. Фиксация заголовка

Сохранение параметров поездок и станций в виде списка, может пригодиться далее

In [10]:
trip_parametr = list(enumerate(headerOfTrips.split(",")))
station_parametr = list(enumerate(headerOfStations.split(",")))

Просмотр результата

In [11]:
trip_parametr

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

In [12]:
station_parametr

[(0, 'id'),
 (1, 'name'),
 (2, 'lat'),
 (3, 'long'),
 (4, 'dock_count'),
 (5, 'city'),
 (6, 'installation_date')]

## Шаг 3. Разбиение данных.

### 3.1. Разбиение данных

Произведем разбиемние данных от строкового формата к ячеечному, для облегченоого доступа к данным

In [13]:
final_trips_data = tripsWithOutHeader.map(lambda x: x.split(','))
final_stations_data = stationsWithOutHeader.map(lambda x: x.split(','))

### 3.2 Просмотр данных

In [14]:
final_trips_data.take(5)

[['4576',
  '63',
  '',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138'],
 ['4130',
  '71',
  '8/29/2013 10:16',
  'Mountain View City Hall',
  '27',
  '8/29/2013 10:17',
  'Mountain View City Hall',
  '27',
  '48',
  'Subscriber',
  '97214'],
 ['4251',
  '77',
  '8/29/2013 11:29',
  'San Jose City Hall',
  '10',
  '8/29/2013 11:30',
  'San Jose City Hall',
  '10',
  '26',
  'Subscriber',
  '95060'],
 ['4299',
  '83',
  '8/29/2013 12:02',
  'South Van Ness at Market',
  '66',
  '8/29/2013 12:04',
  'Market at 10th',
  '67',
  '319',
  'Subscriber',
  '94103']]

In [15]:
final_stations_data.take(5)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013'],
 ['4',
  'Santa Clara at Almaden',
  '37.333988',
  '-121.894902',
  '11',
  'San Jose',
  '8/6/2013'],
 ['5',
  'Adobe on Almaden',
  '37.331415',
  '-121.8932',
  '19',
  'San Jose',
  '8/5/2013'],
 ['6',
  'San Pedro Square',
  '37.336721000000004',
  '-121.894074',
  '15',
  'San Jose',
  '8/7/2013']]

# Этап 3. Разработка и применение модели.

## Шаг 1. Модель для Trips

Для описания модели обратимся к зафиксированному заголовку данных на шаге 2.3 Этапа 1

In [16]:
trip_parametr

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

На основании этих данных и будем строить модель Trips

In [17]:
def initModelTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        try:
            yield Trip(                             
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            ) 
        except:
            pass

## Шаг 2. Модель для Stantions

Для описания модели обратимся к зафиксированному заголовку данных на шаге 2.3 Этапа 1

In [18]:
station_parametr

[(0, 'id'),
 (1, 'name'),
 (2, 'lat'),
 (3, 'long'),
 (4, 'dock_count'),
 (5, 'city'),
 (6, 'installation_date')]

На основании этих данных и будем строить модель Stantions

In [19]:
def initModelStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

## Шаг 3. Применение модели к полученным данным.

### 3.1 Применение модели Trips

In [20]:
tripsProcessedByModel = final_trips_data.mapPartitions(initModelTrip)

#### Вывод результатов

In [21]:
tripsProcessedByModel.take(10)

[Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214'),
 Trip(trip_id=4251, duration=77, start_date=datetime.datetime(2013, 8, 29, 11, 29), start_station_name='San Jose City Hall', start_station_id=10, end_date=datetime.datetime(2013, 8, 29, 11, 30), end_station_name='San Jose City Hall', end_station_id='10', bike_id=26, subscription_type='Subscriber', zip_code='95060'),
 Trip(trip_id=4299, duration=83, start_date=datetime.datetime(2013, 8, 29, 12, 2), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 12, 4), end_station_name='Market at 10th', end_station_id='67', bike_id=319, subscription_type='Subscriber', zip_code='94103'),
 Trip(trip_id=4927, duration=103, s

### 3.2 Применение модели Stations

In [22]:
stationsProcessedByModel = final_stations_data.mapPartitions(initModelStation)

#### Вывод результатов

In [23]:
stationsProcessedByModel.take(10)

[Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
 Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0)),
 Station(station_id=4, name='Santa Clara at Almaden', lat=37.333988, long=-121.894902, dockcount=11, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
 Station(station_id=5, name='Adobe on Almaden', lat=37.331415, long=-121.8932, dockcount=19, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0)),
 Station(station_id=6, name='San Pedro Square', lat=37.336721000000004, long=-121.894074, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 7, 0, 0)),
 Station(station_id=7, name='Paseo de San Antonio', lat=37.333798, long=-121.88694299999999, dockcount=15, landmark='San J

# Этап 4. Выполнение заданий.

## <center> ***Задание 1. Найти велосипед с максимальным временем пробега.*** </center>

### Преобразование данных к виду ключ-значение

Для работы нам потребуется список пар:<br>
`ключ (номер id велосипеда) - значение (данные в строке с этим id)`

In [24]:
tripMapWithBicycle = tripsProcessedByModel.keyBy(lambda x: x.bike_id)
tripMapWithBicycle.take(5)

[(48,
  Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214')),
 (26,
  Trip(trip_id=4251, duration=77, start_date=datetime.datetime(2013, 8, 29, 11, 29), start_station_name='San Jose City Hall', start_station_id=10, end_date=datetime.datetime(2013, 8, 29, 11, 30), end_station_name='San Jose City Hall', end_station_id='10', bike_id=26, subscription_type='Subscriber', zip_code='95060')),
 (319,
  Trip(trip_id=4299, duration=83, start_date=datetime.datetime(2013, 8, 29, 12, 2), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 12, 4), end_station_name='Market at 10th', end_station_id='67', bike_id=319, subscription_type='Subscriber', zip_code='94103')),
 (527,
  T

### Группировка данных по продолжительности (duration)

In [25]:
durationBicycleMap = tripMapWithBicycle.mapValues(lambda x: x.duration).reduceByKey(lambda x1, x2: x1 + x2)

### Определение топ 1 (duration)

In [26]:
topOfDurationBicycleMap = durationBicycleMap.top(1, key=lambda x: x[1])[0][0]

### <center> ***Ответ на задание 1. Найти велосипед с максимальным временем пробега.*** </center>

In [27]:
topOfDurationBicycleMap

535

## <center> ***Задание 2. Найти наибольшее геодезическое расстояние между станциями.*** </center>

### Подключение библиотеки `math`

Данная библиотека необходим для производства математических вичислений при расчете расстояния.

In [28]:
from math import *

### Описание метода для расчета геодезического расстояния

In [29]:
def distanceBetweenStations(gLatitudeFirstStation, gLongitudeFirstStation, gLatitudeSecondStation, gLongitudeSecondStation):
  earth_radius = 6371.0088
  gLatitudeFirstStation, gLongitudeFirstStation = radians(gLatitudeFirstStation), radians(gLongitudeFirstStation)
  gLatitudeSecondStation, gLongitudeSecondStation = radians(gLatitudeSecondStation), radians(gLongitudeSecondStation)

  differenceLatitude = gLatitudeSecondStation - gLatitudeFirstStation
  differenceLongitude = gLongitudeSecondStation - gLongitudeFirstStation
  
  haversinusLatitutde = (sin(differenceLatitude / 2)) ** 2
  haversinusLongitude = (sin(differenceLongitude / 2)) ** 2
  
  return 2 * earth_radius * sqrt(haversinusLatitutde + cos(gLatitudeFirstStation)\
                                 * cos(gLatitudeSecondStation) * haversinusLongitude)

### Вычисление расстояний между станциями

In [31]:
mapDistance = stationsProcessedByModel.cartesian(stationsProcessedByModel)\
                         .filter(lambda x: x[0].station_id != x[1].station_id)\
                         .map(lambda x: [x[0], x[1], distanceBetweenStations(x[0].lat, x[0].long, x[1].lat, x[1].long)])\
                         .keyBy(lambda x: (x[0].name, x[1].name))\
                         .reduce(lambda x1, x2: x1 if x1[1] > x2[1] else x2)

### Определение максимального геодезического расстояния

In [33]:
wayBetweenStantion = mapDistance[0]
kilometrsBetweenStantion = mapDistance[1][2]

### <center> ***Ответ на задание 2.  Найти наибольшее геодезическое расстояние между станциями.*** </center>

In [34]:
print(f"Максимальное расстояние между станциями: {wayBetweenStantion}, {kilometrsBetweenStantion}.")

Максимальное расстояние между станциями: ('Ryland Park', 'Mezes Park'), 34.317914350160784.


## <center> ***Задание 3. Найти путь велосипеда с максимальным временем пробега через станции.*** </center>

In [35]:
wayBicycleWithMaxDuration = tripsProcessedByModel.filter(lambda x: x.bike_id == topOfDurationBicycleMap)\
.sortBy(lambda x: x.start_date).map(lambda x: (x.start_station_name, x.end_station_name))

### <center> ***Ответ на задание 3. Найти путь велосипеда с максимальным временем пробега через станции.*** </center>

In [39]:
wayBicycleWithMaxDuration.take(10)

[('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)'),
 ('San Francisco Caltrain (Townsend at 4th)',
  'San Francisco Caltrain 2 (330 Townsend)'),
 ('San Francisco Caltrain 2 (330 Townsend)', 'Market at Sansome'),
 ('Market at Sansome', '2nd at South Park'),
 ('2nd at Townsend', 'Davis at Jackson'),
 ('San Francisco City Hall', 'Civic Center BART (7th at Market)'),
 ('Civic Center BART (7th at Market)', 'Post at Kearney'),
 ('Post at Kearney', 'Embarcadero at Sansome'),
 ('Embarcadero at Sansome', 'Washington at Kearney'),
 ('Washington at Kearney', 'Market at Sansome')]

## <center> ***Задание 4. Найти количество велосипедов в системе.*** </center>

In [40]:
countOfBicycle = tripsProcessedByModel.map(lambda x: x.bike_id).distinct().count()

### <center> ***Ответ на задание 4. Найти количество велосипедов в системе.*** </center>

In [41]:
countOfBicycle

700

## <center> ***Задание 5. Найти пользователей потративших на поездки более 3 часов.*** </center>

In [48]:
threeHours = 3*60*60
usersOfBicycle = tripsProcessedByModel.filter(lambda x: x.duration > threeHours)\
.map(lambda x: x.zip_code).filter(lambda x: x != "").distinct()

### <center> ***Ответ на задание 5. Найти пользователей потративших на поездки более 3 часов.*** </center>

In [49]:
print('Топ-10 пользователей потративших на поездки более 3 часов = ', usersOfBicycle.take(10))

Топ-10 пользователей потративших на поездки более 3 часов =  ['58553', '94301', '94039', '94133', '93726', '94123', '4517', '29200', '45322', '94080']
